<a href="https://colab.research.google.com/github/Kabongosalomon/masakhane/blob/tshiluba/en-lua/jw300-baseline/salomon_en_lua_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

### Languages: English-Tshiluba

### Author: Salomon KABONGO KABENAMUALU

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "lua" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-lua-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 5.6MB/s eta 0:00:011


In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-lua.xml.gz not found. The following files are available for downloading:

   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-lua.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  32 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/lua.zip

 298 MB Total size
./JW300_latest_xml_en-lua.xml.gz ... 100% of 3 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_lua.zip ... 100% of 32 MB


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-02 08:48:18--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2020-01-02 08:48:18 (16.5 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-02 08:48:22--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-lua.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [7]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [8]:
!ls

drive			     JW300_latest_xml_en.zip   sample_data     test.lua
jw300.en		     JW300_latest_xml_lua.zip  test.en
JW300_latest_xml_en-lua.xml  jw300.lua		       test.en-any.en


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 5835/330019 lines since contained in test set.


source_sentence                                    target_sentence
0                Our Planet ​ — What Is Its Future ?     Nditekemena kayi didiku bua buloba buetu ebu ?
1  “ While humans for millennia have feared ‘ act...  Bilondeshile tshikandakanda kampanda tshia mu ...
2  The United Nations Environment Programme ( UNE...  ( Globe and Mail ) Tshibambalu tshia Ndongamu ...
3  Executive director of UNEP , Klaus Toepfer , s...  Klaus Toepfer , mulombodi wa ndongamu eu udi w...
4  Some environmental progress has been made sinc...  Katshia benza Tshibambalu etshi mu 1972 bua ku...
5  As reported in The Toronto Star , “ the qualit...  Nunku , anu mudi tshinga tshikandakanda tshile...
6  Also , forest management programs , such as th...  Kabidi , ndongamu ya malu a mêtu bu mudi ya mu...
7  Even so , the UNEP report says that if economi...  Nansha nanku , luapolo lua Tshibambalu etshi l...
8  The Globe stated : “ About half the world’s ri...  ( Toronto Star ) Tshinga tshikandakanda tshiak...
9  Eighty countries holding 40 per cent of the wo...  Mu matunga 80 mudi bia pa lukama 40 bia bantu ...

In [10]:
df.source_sentence[100]

'“ A lover of silver will never be satisfied with silver , nor a lover of wealth with income . ”'

In [11]:
df.target_sentence[100]

'“ Muntu udi munange arjan kakukuta arjan nansha , anyi muntu udi munange biuma kakukuta tshidiye upeta to . ”'

In [12]:
len(df)

324185

## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [13]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [14]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144665 sha256=22f887c0a65f3e17aff31af915510aaca660b1c7640faab25f7e06d481010a6c
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.11 0.00 percent complete
00:00:19.82 0.34 percent complete
00:00:39.20 0.68 percent complete
00:00:58.69 1.02 percent complete
00:01:17.74 1.36 percent complete
00:01:36.53 1.70 percent complete
00:01:55.66 2.04 percent complete
00:02:15.04 2.38 percent complete
00:02:34.23 2.72 percent complete
00:02:53.52 3.06 percent complete
00:03:12.97 3.40 percent complete
00:03:32.06 3.74 percent complete
00:03:52.09 4.08 percent complete
00:04:11.10 4.42 percent complete
00:04:30.15 4.76 percent complete
00:04:49.32 5.10 percent complete
00:05:09.39 5.44 percent complete
00:05:28.23 5.78 percent complete
00

00:08:39.05 9.18 percent complete
00:08:58.16 9.52 percent complete
00:09:17.02 9.86 percent complete
00:09:35.73 10.20 percent complete
00:09:54.60 10.54 percent complete
00:10:14.97 10.88 percent complete
00:10:33.72 11.22 percent complete
00:10:52.61 11.56 percent complete
00:11:11.66 11.89 percent complete
00:11:30.51 12.23 percent complete
00:11:49.01 12.57 percent complete
00:12:07.46 12.91 percent complete
00:12:26.61 13.25 percent complete
00:12:46.55 13.59 percent complete


00:13:05.73 13.93 percent complete
00:13:24.66 14.27 percent complete
00:13:43.44 14.61 percent complete
00:14:02.20 14.95 percent complete
00:14:20.63 15.29 percent complete
00:14:39.87 15.63 percent complete
00:14:58.84 15.97 percent complete
00:15:19.03 16.31 percent complete
00:15:38.00 16.65 percent complete
00:15:57.15 16.99 percent complete
00:16:16.45 17.33 percent complete
00:16:35.21 17.67 percent complete
00:16:53.90 18.01 percent complete
00:17:12.40 18.35 percent complete
00:17:31.51 18.69 percent complete
00:17:51.08 19.03 percent complete
00:18:09.93 19.37 percent complete
00:18:28.96 19.71 percent complete
00:18:47.58 20.05 percent complete
00:19:06.15 20.39 percent complete
00:19:25.12 20.73 percent complete
00:19:44.31 21.07 percent complete
00:20:03.58 21.41 percent complete
00:20:24.21 21.75 percent complete
00:20:42.98 22.09 percent complete
00:21:01.78 22.43 percent complete
00:21:21.01 22.77 percent complete
00:21:40.49 23.11 percent complete
00:21:59.30 23.45 pe

00:33:09.44 35.00 percent complete
00:33:28.66 35.34 percent complete
00:33:49.08 35.68 percent complete
00:34:08.21 36.02 percent complete
00:34:28.07 36.36 percent complete
00:34:47.97 36.70 percent complete
00:35:07.64 37.04 percent complete


00:35:27.34 37.38 percent complete
00:35:47.06 37.72 percent complete
00:36:07.03 38.06 percent complete
00:36:28.11 38.40 percent complete
00:36:47.71 38.74 percent complete
00:37:07.12 39.08 percent complete
00:37:26.57 39.42 percent complete
00:37:46.25 39.76 percent complete
00:38:06.24 40.10 percent complete
00:38:25.59 40.44 percent complete
00:38:45.46 40.78 percent complete
00:39:06.48 41.12 percent complete
00:39:26.11 41.46 percent complete
00:39:46.05 41.80 percent complete
00:40:05.21 42.14 percent complete
00:40:24.72 42.48 percent complete
00:40:44.27 42.82 percent complete
00:41:03.51 43.16 percent complete
00:41:22.69 43.50 percent complete
00:41:44.08 43.84 percent complete
00:42:03.53 44.18 percent complete
00:42:23.17 44.52 percent complete
00:42:42.10 44.86 percent complete
00:43:01.06 45.20 percent complete
00:43:20.28 45.54 percent complete
00:43:39.35 45.88 percent complete
00:43:58.47 46.22 percent complete
00:44:19.55 46.56 percent complete
00:44:39.13 46.90 pe

...

01:03:29.10 66.27 percent complete
01:03:48.20 66.61 percent complete
01:04:07.81 66.95 percent complete
01:04:27.66 67.29 percent complete
01:04:47.38 67.63 percent complete
01:05:07.33 67.97 percent complete
01:05:26.41 68.31 percent complete
01:05:47.61 68.65 percent complete
01:06:06.71 68.99 percent complete


01:06:26.54 69.33 percent complete
01:06:47.00 69.67 percent complete
01:07:06.34 70.01 percent complete
01:07:26.26 70.35 percent complete
01:07:45.25 70.69 percent complete
01:08:05.32 71.03 percent complete
01:08:26.28 71.37 percent complete
01:08:45.19 71.71 percent complete
01:09:03.98 72.05 percent complete
01:09:24.04 72.39 percent complete
01:09:43.90 72.73 percent complete
01:10:03.51 73.07 percent complete
01:10:22.93 73.41 percent complete
01:10:42.70 73.75 percent complete
01:11:03.95 74.09 percent complete
01:11:22.98 74.43 percent complete
01:11:42.22 74.77 percent complete
01:12:01.77 75.11 percent complete
01:12:22.58 75.45 percent complete
01:12:41.59 75.79 percent complete
01:13:01.23 76.13 percent complete
01:13:21.02 76.47 percent complete
01:13:41.38 76.81 percent complete
01:14:00.23 77.15 percent complete
01:14:19.54 77.49 percent complete
01:14:38.85 77.83 percent complete
01:14:59.10 78.17 percent complete
01:15:19.31 78.51 percent complete
01:15:38.36 78.85 pe

01:33:31.97 97.20 percent complete
01:33:51.65 97.54 percent complete
01:34:11.63 97.88 percent complete


01:34:31.02 98.22 percent complete
01:34:51.22 98.56 percent complete
01:35:12.27 98.90 percent complete
01:35:31.84 99.24 percent complete
01:35:51.85 99.58 percent complete


01:36:11.29 99.92 percent complete


In [15]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
He was orderly , industrious , patient , and protective of all those under his care .
After giving a thorough witness , he said to the conscience - stricken Jews : “ Repent , and let each one of you be baptized in the name of Jesus Christ for forgiveness of your sins . ”
In the original Greek , the word translated “ readjust ” can also refer to the realignment or setting of broken bones , a painful procedure .
Why are so many today in need of “ a strengthening aid ” ?
As we can see from the case of the Mount Fugen eruption , we need to act on the warning .
Submitting to Jesus ’ teachings as his disciples will give our life real meaning and satisfaction .
What makes a woman truly praiseworthy ?
“ I can’t simply erase memories of my first marriage , especially when we travel to the same places where my ex - husband and I went on vacation , ” says Ellen , who lives in South Africa .
It took me a while to see how all the pieces fit together too .
How did Jehovah react ?

=



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [17]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 7.7MB/s 
     |████████████████████████████████| 307kB 13.2MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 215kB 21.6MB/s 
     |████████████████████████████████| 737kB 19.9MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=2d995a2a774d7ca37f210debdc7b722704c02354dbfab15bbae9aec40d82a4ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-oqdckzxn/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml: filename=PyYAML-5.2-cp36-cp36m-linux_x86_64.whl size=44209 sha256=b3c4b27e927be9e22911b8d7cd5f57a071b26c84561317d031ee2486ffe701d3
  Stored in directory: /root/.cache/pip/wheels/54/b7/c7/2ada654ee54483c9329871665aaf4a6056c3ce36f29cf66e67
Successfully bui

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [18]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Tshiluba Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.lua    test.lua	    train.en
dev.bpe.en	dev.lua      test.en	     train.bpe.en   train.lua
dev.bpe.lua	test.bpe.en  test.en-any.en  train.bpe.lua
bpe.codes.4000	dev.en	     test.bpe.lua    test.en-any.en.1  train.bpe.lua
dev.bpe.en	dev.lua      test.en	     test.lua	       train.en
dev.bpe.lua	test.bpe.en  test.en-any.en  train.bpe.en      train.lua
BPE Tshiluba Sentences
N@@ di@@ sungula kayi didi yonso wa kutudi ne bua kuenza ?
( Bala Nsumuinu 14 : 5 ; Efeso 4 : 25 . )
Badi bapeta “ nyuma ” bu bantu badi ‘ bin@@ dile ne muoyo mujima bua Nzambi ab@@ angate bu bana , ba@@ pi@@ kudi@@ bue ku mi@@ bidi yabu . ’
( Bala Lomo 4 : 20 - 22 . )
( Bala Bienzedi 18 : 2 - 4 ; 20 : 20 , 21 , 34 , 35 . )
Combined BPE Vocab
xt
›
ʼ
ḥ
ondeshile
Â
Î
ʺ
œ@@
ansha


In [19]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.lua    test.en-any.en.1  train.bpe.lua
dev.bpe.en	dev.lua      test.en	     test.lua	       train.en
dev.bpe.lua	test.bpe.en  test.en-any.en  train.bpe.en      train.lua


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [21]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-01-02 10:37:34,915 Hello! This is Joey-NMT.
2020-01-02 10:37:36,732 Total params: 12164864
2020-01-02 10:37:36,733 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [30]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-lua-baseline/models/enlua_transformer/best.ckpt': Operation not supported


In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp joeynmt/models/${src}${tgt}_transformer/best.ckpt "$gdrive_path/models/${src}${tgt}_transformer/"

In [32]:
!ls joeynmt/models/${src}${tgt}_transformer

00098000.hyps.dev   28000.hyps	48000.hyps  68000.hyps	88000.hyps
00098000.hyps.test  29000.hyps	49000.hyps  69000.hyps	89000.hyps
10000.hyps	    30000.hyps	50000.hyps  70000.hyps	90000.hyps
1000.hyps	    3000.hyps	5000.hyps   7000.hyps	9000.hyps
11000.hyps	    31000.hyps	51000.hyps  71000.hyps	91000.hyps
12000.hyps	    32000.hyps	52000.hyps  72000.hyps	92000.hyps
13000.hyps	    33000.hyps	53000.hyps  73000.hyps	93000.hyps
14000.hyps	    34000.hyps	54000.hyps  74000.hyps	94000.hyps
15000.hyps	    35000.hyps	55000.hyps  75000.hyps	95000.hyps
16000.hyps	    36000.hyps	56000.hyps  76000.hyps	96000.ckpt
17000.hyps	    37000.hyps	57000.hyps  77000.hyps	96000.hyps
18000.hyps	    38000.hyps	58000.hyps  78000.hyps	97000.ckpt
19000.hyps	    39000.hyps	59000.hyps  79000.hyps	97000.hyps
20000.hyps	    40000.hyps	60000.hyps  80000.hyps	98000.ckpt
2000.hyps	    4000.hyps	6000.hyps   8000.hyps	98000.hyps
21000.hyps	    41000.hyps	61000.hyps  81000.hyps	best.ckpt
22000.hyps	    42000.hyps	62000.hyps  

In [33]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 102899.78906	PPL: 53.84991	bleu: 1.12929	LR: 0.00030000	*
Steps: 2000	Loss: 84754.05469	PPL: 26.66253	bleu: 2.72745	LR: 0.00030000	*
Steps: 3000	Loss: 75980.53906	PPL: 18.97998	bleu: 5.21574	LR: 0.00030000	*
Steps: 4000	Loss: 70149.67969	PPL: 15.14250	bleu: 7.78372	LR: 0.00030000	*
Steps: 5000	Loss: 66669.04688	PPL: 13.23242	bleu: 9.80320	LR: 0.00030000	*
Steps: 6000	Loss: 62869.32031	PPL: 11.42123	bleu: 11.31751	LR: 0.00030000	*
Steps: 7000	Loss: 60245.42188	PPL: 10.31736	bleu: 12.77085	LR: 0.00030000	*
Steps: 8000	Loss: 58000.27344	PPL: 9.45793	bleu: 14.21334	LR: 0.00030000	*
Steps: 9000	Loss: 56034.40234	PPL: 8.76440	bleu: 15.74813	LR: 0.00030000	*
Steps: 10000	Loss: 54509.39453	PPL: 8.26163	bleu: 16.10168	LR: 0.00030000	*
Steps: 11000	Loss: 53012.06641	PPL: 7.79605	bleu: 17.35481	LR: 0.00030000	*
Steps: 12000	Loss: 51737.62109	PPL: 7.42050	bleu: 18.25946	LR: 0.00030000	*
Steps: 13000	Loss: 50633.38281	PPL: 7.10977	bleu: 18.31056	LR: 0.00030000	*
Steps: 14000	Loss:

In [34]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-01-02 14:49:38,856 Hello! This is Joey-NMT.
2020-01-02 14:50:04,491  dev bleu:  29.96 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-01-02 14:50:37,607 test bleu:  42.52 [Beam search decoding with beam size = 5 and alpha = 1.0]
